In [1]:
## packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'seaborn'

# Practical session 5: missing data, scaling and dimensionality reduction

<div class="alert alert-block alert-info">
    <h2>Table of Contents </h2><a class="anchor" id="top"></a>
    <br><a href="#section_1"> Missing data</a>
    <br><a href="#intermezzo">Intermezzo - Scaling </a>
    <br><a href="#section_2"> Dimensionality reduction </a>
    <br><a href="#Optional">  Optional - Clustering</a>
</div>

<div class="alert alert-block alert-info"> 
 In this practical we will cover how to handle missing data, normalization and dimensionality reduction. At the end of a practical there is an optional part about cluster analysis for those interested. Clustering algorithms are implemented in scikit-learn very similarly as classification or regression algorithms. So you should be able to solve this on your own quite easily. 
 
 
 Last theory lecture also included hyperparameter optimization, but this was already covered during the practical class of Week 3.
</div>

<h2 style="display: inline"> Missing data </h2> <span style="float: right"><a href="#top">[back to top]</a></span> <a class="anchor" id="section_1"></a> 

<div class="alert alert-block alert-warning">
For reproducibility purposes we will always use random_state=0 in this practical. Don't forget to assign this parameter in your functions if it is needed.
</div>

For this part of the practical we will work with the *communities* dataset, which has a large number of features, as well as missing values. It contains demographic information about countries in the United States, and the goal is to predict the number of violent crimes (attribute A122).

In [ ]:
# We load the dataset and extract the feature we want to predict
data_dir = "/data/gent/shared/000/gvo00070/BigDataScience2023/"
df = pd.read_csv(data_dir + 'communities.psv', sep = '|', na_values = '?')
X = df.drop("A122", axis=1)
y = df["A122"]

<div class="alert alert-success">
<h3> Exercise:</h3>
<ol> 
 <li>Divide the dataset into a training (80%) and test (20%) set
 <li>For the training set, count the number of attributes with missing values as well as the percentage of missing values
</ol> 

**HINT**: look at how the missing values are represented in the data (their datatype)
</div>

In [ ]:
# Perform the train-test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
# Count the number of attributes with missing values
attributes_with_missing = np.sum(np.any(np.isnan(X_train), axis=0))
print("Attributes with missing values:", attributes_with_missing)

In [ ]:
## Calculate the percentage of missing values
missing_rate = np.count_nonzero(np.isnan(X_train))/np.prod(X_train.shape)
print("Percentage of missing values:", missing_rate*100)

<div class="alert alert-success">
<h3> Exercise:</h3>
There are multiple single imputation strategies for missing data implemented in scikit-learn. Perform the following strategies:
<ol> 
 <li><a href="https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html">Simple imputation</a> with the mean
 <li><a href="https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html">Simple imputation</a> with the median
 <li><a href="https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html#sklearn.impute.KNNImputer">Nearest Neighbor imputation</a> 
</ol> 
Fit for each strategy a <a href="https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html">linear regression model </a>and evaluate the model's performance with the R-squared metric.

Each of these methods has a parameter <i>add_indicator</i>. Find out what the purpose of this parameter is and evaluate it's influence on the prediction performance.
</div>

The *add_indicator* parameters allows to use information about which values are missing as an extra feature. For a single array this would mean that the algorithm would add a column to the data that indicates missingness of a data point in the non-transformed data.

In [ ]:
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.linear_model import LinearRegression

# Declare the imputation methods
imputation_methods_no_ind = {
    'mean imputation': SimpleImputer(strategy="mean", add_indicator=False),
    'median imputation': SimpleImputer(strategy="median", add_indicator=False),
    'neighbours imputation': KNNImputer( add_indicator=False),
}
imputation_methods_ind = {
    'mean imputation': SimpleImputer(strategy="mean", add_indicator=True),
    'median imputation': SimpleImputer(strategy="median", add_indicator=True),
    'neighbours imputation': KNNImputer( add_indicator=True),
}

# Declare the model
clf = LinearRegression()

# Run the imputators + models
## Without inidicators
scores_no_ind = np.zeros(3)
methods = []
i = 0
for method, imputer in imputation_methods_no_ind.items():
    imputer.fit(X_train)
    X_train_imp = imputer.transform(X_train)
    X_test_imp = imputer.transform(X_test)
    
    clf.fit(X_train_imp, y_train)
    scores_no_ind[i] = clf.score(X_test_imp, y_test)
    methods.append(method)
    i += 1
    
## With indicators
scores_ind = np.zeros(3)
i = 0
for method, imputer in imputation_methods_ind.items():
    imputer.fit(X_train)
    X_train_imp = imputer.transform(X_train)
    X_test_imp = imputer.transform(X_test)
    
    clf.fit(X_train_imp, y_train)
    scores_ind[i] = clf.score(X_test_imp, y_test)
    i += 1

# Plot the results
fig, ax = plt.subplots()
ax.scatter(methods, scores_ind,  label = 'With indicator' )
ax.scatter(methods, scores_no_ind,  label = 'Without indicator' )
ax.legend()
plt.ylabel('Coefficient of determination')
plt.show()

<h2 style="display: inline"> Intermezzo - Scaling </h2> <span style="float: right"><a href="#top">[back to top]</a></span> <a class="anchor" id="intermezzo"></a> 

As mentioned in the lectures, scaling/normalization is oftentimes quite important as it can heavily influence prediction results. Therefore,  **it is almost always recommended to perform some form of normalization or scaling of the data**, especially when regularization, distance-based classifiers or classifiers with normality/distribution assumptions are implemented. In scikit-learn you have the <a href="https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html">StandardScaler</a> and <a href="https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler">RobustScaler</a> available for respectively standardizaton and robust feature scaling. 

But do note, that in order to prevent data leakage, **scaling can only be based on the training dataset characteristics**. As the model is trained on scaled features, for a proper prediction the test data will also need to be on the same scale. But the test data can only be scaled based on the training data's characteristics, not on its own characteristics. *E.g. for standardization, the mean and variance of the training data will be calculated and used for transformation of the entire dataset.*

Worthwhile to mention here is also the <a href="https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html"> pipeline constructor </a> in scikit-learn. This constructor allows you to sequentially perform multiple scikit-learn transform actions and one final estimation action.

So for instance say you would want to perform a scaling transformation and then fit a linear SVM model, with the help of the pipeline constructor this can be efficiently implemented:

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.pipeline import Pipeline

# Construct a toy classification dataset
X, y = make_classification(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=0)

# Construct a pipeline with a scaler and a svc step
pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])  

# This pipeline can be used as any other constructor in scikit-learn
pipe.fit(X_train, y_train)

# The pipeline will inherit the scoring method from the final estimator
# Scaling will only be trained on the train dataset, but the test data will also be transformed
pipe.score(X_test, y_test) 

In [ ]:
# You can access the independent steps
pipe['scaler'].mean_

<h2 style="display: inline"> Dimensionality reduction </h2> <span style="float: right"><a href="#top">[back to top]</a></span> <a class="anchor" id="section_2"></a> 

Dimensionality reduction is an important tool in data science, it performs a transformation of data from a high-dimensional space into a low-dimensional space so that the low-dimensional representation retains some meaningful properties of the original data. These properties vary with each algorithm.

**Principal component analysis (PCA)** might be the most known dimensionality reduction method. It's goal is to project the data to a lower dimension coordinate system by preserving as much variance in the data as possible. The (new) axes of the lower dimensional data are then called principal components (PCs)

**Linear Discriminant analysis** (introduced during week 3 as a classifier) can also be used as a dimensionality reduction method, but its objective for dimension reduction is not maximizing variance, but finding the lower dimensional space that best separates the classes present in the data.

![CV](./PCA_LDA.png)

Many other exist, such as **UMAP**,  **t-SNE**, **MDS** etc.

Dimensionality reduction is used within data science for two purposes:
<ol>
    <li> <i>Data visualisation</i>
    <li> <i>Feature reduction</i>
</ol>


For this part of the practical we will make use of *the Winconsin Diagnostic Breast Cancer (wdbc)* dataset that can be used to classify tumors as benign or malignant.

In [ ]:
data_dir = "/data/gent/shared/000/gvo00070/BigDataScience2023/"
df = pd.read_csv(data_dir + 'wdbc.psv', sep = '|', na_values = '?')
X = df.drop('class', axis=1)
y = df['class']

In [ ]:
X

<div class="alert alert-success">
<h3> Exercise:</h3>
<ol>
    <li> Perform <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html">5-fold CV</a>
    <li> Scale the data 
    <li> Perform a <a href="https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html">PCA</a> transformation and make sure 85% of the variation is retained in the lower dimension
    <li> Classify the data with a <a href="https://sci>kit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html">Decision tree classifier</a>
    <li> Evaluate the model with the mean <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score">accuracy score</a>
    <li> Plot the first PCs against each other colored by class label (<b>HINT</b>: you can access the fitted objects in the pipeline)
<ol>
</div>

In [ ]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, roc_auc_score

# Declare the K-fold constructor
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# initialize an array to store the accuracy per fold
acc = np.zeros(5)

# Perform 5-fold CV
for i, (train_index, test_index) in enumerate(kf.split(X)):
    # Per split, construct the training and test data
    X_train = X.values[train_index]
    X_test= X.values[test_index]
    y_train = y.values[train_index]
    y_test = y.values[test_index]
    
    # Declare the pipeline constructor
    pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('dimensionality_reduction', PCA(n_components=0.85)),
    ('classification', DecisionTreeClassifier()),
    ]) 
    
    # Fit and evaluate the pipeline
    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)
    scores = pipe.predict_proba(X_test)

    # Measure and store the accuracy per fold
    acc[i] = accuracy_score(y_test, preds)


print('mean accuracy score', np.mean(acc))


In [ ]:
# For this part we will use the model constructed in the last fold that is still present in working environment of the notebook
# Scale the training data
X_train_scaled = pipe['scaler'].transform(X_train)

# Extract the first three dimensions of the transformed training data
pc_0 = pipe['dimensionality_reduction'].transform(X_train_scaled)[:,0]
pc_1 = pipe['dimensionality_reduction'].transform(X_train_scaled)[:,1]
pc_2 = pipe['dimensionality_reduction'].transform(X_train_scaled)[:,2]

# Scale the test data
X_test_scaled = pipe['scaler'].transform(X_test)

# Extract the first three dimensions of the transformed test data
pc_0_test = pipe['dimensionality_reduction'].transform(X_test_scaled)[:,0]
pc_1_test = pipe['dimensionality_reduction'].transform(X_test_scaled)[:,1]
pc_2_test = pipe['dimensionality_reduction'].transform(X_test_scaled)[:,2]

# Indicate a coloring dictionary
color = {
    'M': 'red',
    'B': 'blue'
}
color_groups = [color[i] for i in y_train]
color_groups_test = [color[i] for i in y_test]

# Plot
fig, ax = plt.subplots(1,2, figsize = (15,5))
ax[0].scatter(pc_0, pc_1, c = color_groups)
ax[0].set_title('PC0-PC1 Train')
ax[0].set_xlabel('PC 0')
ax[0].set_ylabel('PC 1')

ax[1].scatter(pc_0_test, pc_1_test, c = color_groups_test)
ax[1].set_title('PC0-PC1 Test')
ax[1].set_xlabel('PC 0')
ax[1].set_ylabel('PC 1')

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (15,5))
ax[0].scatter(pc_1, pc_2, c = color_groups)
ax[0].set_title('PC1-PC2 Train')
ax[0].set_xlabel('PC 1')
ax[0].set_ylabel('PC 2')

ax[1].scatter(pc_1_test, pc_2_test, c = color_groups_test)
ax[1].set_title('PC1-PC2 Test')
ax[1].set_xlabel('PC 1')
ax[1].set_ylabel('PC 2')

<h2 style="display: inline"> Optional - Clustering </h2> <span style="float: right"><a href="#top">[back to top]</a></span> <a class="anchor" id="Optional"></a> 

<div class="alert alert-success">
<h3> Exercise:</h3>
<ol>
    <li> Split the data in a training (80%) and test (20%) set
    <li> Classify the data with a <a href="https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans">Kmeans</a> and test out some different <i>n_cluster</i> parameters 
    <li> Visualize the predictions of the different models
<ol>
</div>

In [ ]:
# Generate some toy data
from sklearn.datasets import make_blobs
X, y = make_blobs(n_samples=5000,
  centers=5,
  cluster_std=2,
  random_state=42
 )
plt.scatter(X[:,0], X[:,1], c = y)

In [ ]:
from sklearn.cluster import KMeans

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Declare the n_cluster paramters 
params_n_clusters = [2,3,4,5,6,7]

# initialize the figure and make the individual axis easily accessible in the for loop
fig, axes = plt.subplots(2,3, figsize=(15,10))
coords = [[0,0], [0,1], [0,2], [1,0], [1,1], [1,2]]

# loop over the parameters, fit the model, predict the classes and visualize the predictions in a subplot
for i in range(0,len(params_n_clusters)):
    clf = KMeans(n_clusters = params_n_clusters[i])
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    
    ax = axes[coords[i][0], coords[i][1]]
    ax.scatter(X_test[:,0], X_test[:,1], c = preds)
    ax.set_title('n clusters: ' + str(params_n_clusters[i]) )
    
